In [2]:
%pip install arxiv


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [6]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_core.tools import FunctionTool
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()

api_version = os.getenv("AZURE_OPENAI_API_VERSION")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
azure_openai_chat_completion_client = AzureOpenAIChatCompletionClient(
            model="gpt-4o",
            azure_endpoint=azure_endpoint,
            api_version=api_version,
            api_key=api_key,
)


/home/andy/works/openai/autogen04-sample/.venv/lib/python3.11/site-packages/autogen_ext/models/openai/_openai_client.py:1029: UserWarning: Previous behavior of using the model name as the deployment name is deprecated and will be removed in 0.4. Please specify azure_deployment
  client = _azure_openai_client_from_config(copied_args)


In [13]:
def google_search(query: str, num_results: int = 2, max_chars: int = 500) -> list:  # type: ignore[type-arg]
    import os
    import time

    import requests
    from bs4 import BeautifulSoup
    from dotenv import load_dotenv

    load_dotenv()

    serp_api_key = os.getenv("SERPAPI_KEY")

    if not serp_api_key:
        raise ValueError("SerpAPI key not found in environment variables")

    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "num": num_results,
        "api_key": serp_api_key,
    }

    response = requests.get(url, params=params)

    if response.status_code != 200:
        print(response.json())
        raise Exception(f"Error in API request: {response.status_code}")

    results = response.json().get("organic_results", [])

    def get_page_content(url: str) -> str:
        try:
            response = requests.get(url, timeout=10)
            soup = BeautifulSoup(response.content, "html.parser")
            text = soup.get_text(separator=" ", strip=True)
            words = text.split()
            content = ""
            for word in words:
                if len(content) + len(word) + 1 > max_chars:
                    break
                content += " " + word
            return content.strip()
        except Exception as e:
            print(f"Error fetching {url}: {str(e)}")
            return ""

    enriched_results = []
    for item in results:
        body = get_page_content(item["link"])
        enriched_results.append(
            {"title": item["title"], "link": item["link"], "snippet": item.get("snippet", ""), "body": body}
        )
        time.sleep(1)  # Be respectful to the servers

    print(enriched_results)    

    return enriched_results


In [12]:
def arxiv_search(query: str, max_results: int = 2) -> list:  # type: ignore[type-arg]
    """
    Search Arxiv for papers and return the results including abstracts.
    """
    import arxiv

    client = arxiv.Client()
    search = arxiv.Search(query=query, max_results=max_results, sort_by=arxiv.SortCriterion.Relevance)

    results = []
    for paper in client.results(search):
        results.append(
            {
                "title": paper.title,
                "authors": [author.name for author in paper.authors],
                "published": paper.published.strftime("%Y-%m-%d"),
                "abstract": paper.summary,
                "pdf_url": paper.pdf_url,
            }
        )

    # # Write results to a file
    # with open('arxiv_search_results.json', 'w') as f:
    #     json.dump(results, f, indent=2)
    print(results)
    return results

In [7]:
google_search_tool = FunctionTool(
    google_search, description="Search Google for information, returns results with a snippet and body content"
)
arxiv_search_tool = FunctionTool(
    arxiv_search, description="Search Arxiv for papers related to a given topic, including abstracts"
)

In [8]:
google_search_agent = AssistantAgent(
    name="Google_Search_Agent",
    tools=[google_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Google for information, returns results with a snippet and body content",
    system_message="You are a helpful AI assistant. Solve tasks using your tools.",
)

arxiv_search_agent = AssistantAgent(
    name="Arxiv_Search_Agent",
    tools=[arxiv_search_tool],
    model_client=azure_openai_chat_completion_client,
    description="An agent that can search Arxiv for papers related to a given topic, including abstracts",
    system_message="You are a helpful AI assistant. Solve tasks using your tools. Specifically, you can take into consideration the user's request and craft a search query that is most likely to return relevant academi papers.",
)


report_agent = AssistantAgent(
    name="Report_Agent",
    model_client=azure_openai_chat_completion_client,
    description="Generate a report based on a given topic",
    system_message="You are a helpful assistant. Your task is to synthesize data extracted into a high quality literature review including CORRECT references. You MUST write a final report that is formatted as a literature review with CORRECT references.  Your response should end with the word 'TERMINATE'",
)

In [9]:
termination = TextMentionTermination("TERMINATE")
team = RoundRobinGroupChat(
    participants=[google_search_agent, arxiv_search_agent, report_agent], termination_condition=termination
)

In [16]:
await Console(
    team.run_stream(
        # task="멀티 에이전트 AI 시스템 구축을 위한한 논문 리뷰 리포트 작성",
        task="최신 RAG 기술에 대한 동향 및 논문 리뷰",
    )
)

---------- user ----------
최신 RAG 기술에 대한 동향 및 논문 리뷰
---------- Google_Search_Agent ----------
[FunctionCall(id='call_Kts8sCafod2fGJzziWGpXpdO', arguments='{"query": "current trends in RAG technology", "num_results": 2}', name='google_search'), FunctionCall(id='call_EUHkCPGhBjELZbwqquPGwLA9', arguments='{"query": "recent papers on Retrieval-Augmented Generation (RAG)", "num_results": 2}', name='google_search')]
[Prompt tokens: 6590, Completion tokens: 69]
---------- Google_Search_Agent ----------
[FunctionExecutionResult(content="[{'title': 'Latest Developments in Retrieval-Augmented Generation', 'link': 'https://celerdata.com/glossary/latest-developments-in-retrieval-augmented-generation', 'snippet': 'The future of RAG looks promising with continued advancements in technology, model architectures, and training methodologies driving innovation ...', 'body': 'Latest Developments in Retrieval-Augmented Generation PRODUCTS PRODUCTS CelerData Overview CelerData Cloud BYOC SOLUTIONS USE CASE

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='최신 RAG 기술에 대한 동향 및 논문 리뷰', type='TextMessage'), ToolCallMessage(source='Google_Search_Agent', models_usage=RequestUsage(prompt_tokens=6590, completion_tokens=69), content=[FunctionCall(id='call_Kts8sCafod2fGJzziWGpXpdO', arguments='{"query": "current trends in RAG technology", "num_results": 2}', name='google_search'), FunctionCall(id='call_EUHkCPGhBjELZbwqquPGwLA9', arguments='{"query": "recent papers on Retrieval-Augmented Generation (RAG)", "num_results": 2}', name='google_search')], type='ToolCallMessage'), ToolCallResultMessage(source='Google_Search_Agent', models_usage=None, content=[FunctionExecutionResult(content="[{'title': 'Latest Developments in Retrieval-Augmented Generation', 'link': 'https://celerdata.com/glossary/latest-developments-in-retrieval-augmented-generation', 'snippet': 'The future of RAG looks promising with continued advancements in technology, model architectures, and training methodo